In [1]:
cd

/home/robotics


In [2]:
cd Projects/Point-Clouds/

/home/robotics/Projects/Point-Clouds


In [3]:
import open3d as o3d
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:
def detect_planes(points_o3d): #list of all planes, as obox objects, given od3 pointcloud

    points_o3d = points_o3d.voxel_down_sample(voxel_size=0.01) # downsample pointcloud
    points_o3d.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30)) # calcluate normals for each point
    assert (points_o3d.has_normals()) #check if normals exist, else rise error

    #detect planar patches
    planes = points_o3d.detect_planar_patches(
        normal_variance_threshold_deg=30,
        coplanarity_deg=85,
        outlier_ratio=1,
        min_plane_edge_length=0,
        min_num_points=0,
        search_param=o3d.geometry.KDTreeSearchParamKNN(knn=30))
    
    return planes

def get_center_of_plane_of_interest(plane_of_interest): #get center of a single plane
    center = plane_of_interest.get_center()
    return center

def get_corners_of_plane_of_interest(plane_of_interest): #get the list of corners of a single plane, each element in the list is the corner as a np array
    center_of_plane_of_interest = get_center_of_plane_of_interest(plane_of_interest)
    R = plane_of_interest.R
    extents = plane_of_interest.extent
    
    # The plane's normal is assumed to be along the axis with the smallest extent
    min_extent_idx = np.argmin(extents)
    indices = [0, 1, 2]
    indices.remove(min_extent_idx)
    
    # Compute the four corners of the plane
    corners_of_plane_of_interest = []
    for i in [-1, 1]:
        for j in [-1, 1]:
            corner_vect = np.zeros(3)
            corner_vect[indices[0]] = extents[indices[0]] / 2 * i
            corner_vect[indices[1]] = extents[indices[1]] / 2 * j
            # Ensure the corners are on the plane by making the component along the normal very small or zero
            corner_vect[min_extent_idx] = 0  # Place the corner on the plane, assuming the plane is at the center
            corner = center_of_plane_of_interest + R.dot(corner_vect)
            corners_of_plane_of_interest.append(corner)
    
    # Convert each NumPy array corner to a list
    corners_of_plane_of_interest = [corner.tolist() for corner in corners_of_plane_of_interest]

    return corners_of_plane_of_interest

def get_center_of_plane_of_interest(self, plane_of_interest): #get center of a single plane
        center = plane_of_interest.get_center()
        return center

In [5]:
file_name='mani_lidar.ply'
pcd=o3d.io.read_point_cloud(file_name)
coord_frame=o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.5, origin=[0, 0, 0])
o3d.visualization.draw_geometries([pcd,coord_frame])

In [6]:
all_planes = detect_planes(pcd) # list of all planes, as obox objects, given o3d pointcloud


In [7]:
# Create a nested list to store corners of all planes
all_corners = []

for plane in all_planes:
    corners = get_corners_of_plane_of_interest(plane)
    all_corners.append(corners)


# Output the corners of each plane
print(len(all_corners),'planes detected')
print('Corners of each plane:')
for i, corners in enumerate(all_corners, 1):
    print('Corners of plane', i, '=', corners)

7 planes detected
Corners of each plane:
Corners of plane 1 = [[-0.22471723524259707, 0.1907489596479036, -0.8354654407252566], [0.49383788554304603, -0.14844670078267505, -0.7133567350358981], [-0.4701572978226834, -0.23342713291451075, -0.5694440807090698], [0.24839782296295965, -0.5726227933450894, -0.4473353750197113]]
Corners of plane 2 = [[-0.0046974108828948605, 0.2244872091624739, -0.9308042725339686], [0.10598556536167063, 0.20537985712049506, -0.9265510709356225], [-0.03700093577381332, 0.060729219659808806, -0.8258318858084531], [0.07368204047075216, 0.041621867617829975, -0.821578684210107]]
Corners of plane 3 = [[0.25438774988072954, 1.4645184769579913, -2.712588154048568], [0.8172356346664061, 0.07554979904768738, -1.9541916782840734], [-1.6596275163413652, 0.942905371403878, -2.2474024764875224], [-1.0967796315556888, -0.446063306506426, -1.489006000723028]]
Corners of plane 4 = [[1.5670540161315523, 0.88346870755812, -2.500109100747301], [1.9912774211714594, 0.609828666

In [8]:
def visualize_corners_in_pointcloud(pcd, all_corners):
    # Create a list to store corner points
    corner_points = []

    # Flatten the nested list of corners
    for corners in all_corners:
        corner_points.extend(corners)

    # Convert the corner points to a NumPy array
    corner_points_np = np.array(corner_points)

    # Create a point cloud for corner points
    corners_pcd = o3d.geometry.PointCloud()
    corners_pcd.points = o3d.utility.Vector3dVector(corner_points_np)

    # Visualize the point cloud and corner points
    return corners_pcd

In [12]:
def get_all_centers_of_detected_planes(all_planes):
    centers = []
    for plane in all_planes:
        center = get_center_of_plane_of_interest(plane)
        centers.append(center)
    return centers

In [13]:
corners_pcd = visualize_corners_in_pointcloud(pcd,all_corners)

In [14]:
o3d.visualization.draw_geometries([pcd,corners_pcd]+all_planes)


In [20]:
all_centers = get_all_centers_of_detected_planes(all_planes)
print("Centers of all detected planes:", all_centers)



Centers of all detected planes: [array([ 0.01184029, -0.19093692, -0.64140041]), array([ 0.03449231,  0.13305454, -0.87619148]), array([-0.42119594,  0.50922759, -2.10079708]), array([ 1.32431472,  0.18612711, -2.08454491]), array([ 0.11106989,  0.18821464, -0.75237524]), array([ 0.42037051,  0.31181159, -1.18557708]), array([ 0.12473058,  0.01038151, -0.85203864])]


In [23]:
def create_center_point_cloud(all_centers):
    # Convert centers list to NumPy array
    centers_np = np.array(all_centers)

    # Create a PointCloud object from the centers
    center_pcd = o3d.geometry.PointCloud()
    center_pcd.points = o3d.utility.Vector3dVector(centers_np)

    return center_pcd

In [26]:
centers_pcd = create_center_point_cloud(all_centers)
o3d.visualization.draw_geometries([corners_pcd,centers_pcd]+all_planes)

In [70]:
PoI = all_centers[4]
offset = [1/100, 1/100, 1/100]

PoI = [PoI[i] + offset[i] for i in range(len(PoI))]


In [71]:
def get_plane_of_interest(PoI, all_planes):
    all_centers = get_all_centers_of_detected_planes(all_planes)
    
    # Convert PoI and all_centers to numpy arrays for easier computation
    PoI_np = np.array(PoI)
    all_centers_np = np.array(all_centers)
    
    # Calculate the distances between PoI and all_centers
    distances = np.linalg.norm(all_centers_np - PoI_np, axis=1)
    
    # Find the index of the closest center
    closest_index = np.argmin(distances)
    
    # Select the corresponding plane
    plane_of_interest = all_planes[closest_index]
    
    return plane_of_interest


In [72]:
def point_list_to_pointcloud(point_list):
    # Create a PointCloud object
    pointcloud = o3d.geometry.PointCloud()

    # Convert the point list to Open3D Vector3dVector format and assign it to the PointCloud object
    pointcloud.points = o3d.utility.Vector3dVector([point_list])

    return pointcloud

In [73]:
plane_of_interest = get_plane_of_interest(PoI, all_planes)

In [74]:
PoI_pcd = point_list_to_pointcloud(PoI)
PoI_pcd = PoI_pcd.paint_uniform_color([0,0,0])
o3d.visualization.draw_geometries([corners_pcd,centers_pcd,PoI_pcd]+all_planes)
o3d.visualization.draw_geometries([corners_pcd,centers_pcd,PoI_pcd,plane_of_interest])